In [1]:
import datetime as dt
import pandas as pd

import csv
import json
import psaw
import time

In [11]:
def epoch(year, month, day, **kwargs):
    date_time = dt.datetime(year, month, day, **kwargs)
    return int(date_time.timestamp())


def dataframe(psaw_result_generator):
    return pd.DataFrame([item.d_ for item in psaw_result_generator])


# Wrapper subclass to return results in Pandas DataFrames.
class DataframePushshiftAPI(psaw.PushshiftAPI):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def search_comments(self, **kwargs):
        result_gen = super().search_comments(**kwargs)
        return dataframe(result_gen)
    
    def search_submissions(self, **kwargs):
        result_gen = super().search_submissions(**kwargs)
        return dataframe(result_gen)
    
    def search_subreddits(self, **kwargs):
        result_gen = self._search_func(kind='subreddit', **kwargs)
        return dataframe(result_gen)
    
    def redditor_subreddit_activity(self, author, **kwargs):
        result_gen = super().redditor_subreddit_activity(author, **kwargs)
        return dataframe(result_gen)

In [12]:
pushshift = DataframePushshiftAPI()

In [ ]:
def get_subs():
    pushshift.search_subreddits(after=epoch(2017, 1, 1),
                            before=epoch(2018, 1, 1),
                            limit=10)

subs_df = get_subs()

In [ ]:
subs_df